# single-userコンテナ起動確認（Selenium利用）

## 概要

seleniumを利用し、ユーザのログイン～single-user notebook serverコンテナ起動の確認を行います。  

* 前提
    * Moodleにアクセスできること
    * Moodleにテスト用ユーザが登録済みであること
    * Jupyterhubのセットアップが完了しており、MoodleからLTI連携を用いてログインできること
    * seleniumコンテナを利用するノードでDockerが利用可能であること
    * 処理を行うDockerコンテナがMoodle, Jupyterhubにアクセス可能であること
* 実行方法
    * 各セルの説明に従って、実行してください
* 活用方法
    * 指定したユーザ分のコンテナが同時起動可能であることを確認できます。
    * 「801-資源消費量可視化.ipynb」ノートブックを利用して、動作確認中の資源消費量を確認できます。

### UnitGroup名

JupyterHubの構築環境となるVCPのUnitGroup名を指定します。

VCノードを作成時に指定した値を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にして、UnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'JupyterHub'

ugroup_name = 

### チェック

UnitGroup名に対応する group_varsファイルが存在していることを確認します。

In [ ]:
!test -f group_vars/{ugroup_name}

## パラメータの設定

JupyterHubシステムに関する各種設定項目を指定します。

ここで設定された項目はansible変数として保存され、ファイルのmanagerノードへの配置時に各ファイルに反映されます。

### 必須設定項目

lmsに設定したJupyterhubアクセス用のツールのID  
Moodleの場合、JupyterhubにアクセスするURLのクエリパラメータに `id=2` のような形で指定されている。

In [ ]:
lms_tool_id = 

### テスト用ユーザの指定

テスト用ユーザの情報を指定します。  
`edit/`ディレクトリに対象ファイルを保存してください。  
現在使用可能な形式は、`.csv`, `.yaml` です。  
以下のセルを利用し、ファイルを準備してください。

`.csv`で指定する場合のサンプルファイルをコピーします。  
このサンプルファイルを利用するか、以降の`.yaml`形式での設定用のセルを利用して、アカウント情報を登録してください。

In [ ]:
!mkdir -p edit
!cp ../tools/selenium/selenium-client/accounts_sample.csv edit/accounts_sample.csv

アカウント情報を記載するファイル名を指定してください。  
以降のセルを利用して設定する場合は、変更不要です。

In [ ]:
# accounts_file = 'accounts_sample.csv'
accounts_file = 'accounts_file.yaml'

以下の`params`の内容を編集し、アカウント情報を登録してください。  
先に用意した`edit/accounts_sample.csv`や、自身で用意したファイルを利用する場合は、以下のセルを使用する必要はありません。

In [ ]:
import yaml

path = f'edit/accounts_file.yaml'
params = [
    {'username': 'admin', 'password': 'adminpass01'},
]

with open(path, encoding='utf-8', mode="w") as f:
    yaml.safe_dump(params, stream=f)


起動した各ユーザの環境で実行するファイルのファイル名を指定してください。  
ファイルはpythonスクリプトを記述し、`edit`ディレクトリに配置してください。  
`print()`で出力した内容がselenium実行結果の`.json`ファイルに記録されます。

In [ ]:
exec_file = 'test.py'

## Selenium

Selenium実行・処理コンテナに関する設定ファイルのセットアップを行います。

### セットアップ

Selenium実行・処理コンテナに関するセットアップを行います。

まず、ファイルを格納するディレクトリを作成します。

In [ ]:
!mkdir -p tmp/selenium

ファイルを配置します。

In [ ]:
!cp -r ../tools/selenium tmp/
!cp -r edit/{accounts_file} tmp/selenium/
!cp -r edit/{exec_file} tmp/selenium/

設定ファイルの内容を編集します。

In [ ]:
import yaml
from pathlib import Path

with Path('../tools/selenium/docker-compose.yml').open() as f:
    dc_config = yaml.safe_load(f)

gvars_path = Path(f'group_vars/{ugroup_name}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)

dc_config['services']['selenium-client']['volumes'] = [
    {'type': 'bind',
     'source': f'./{accounts_file}',
     'target': f'/app/{accounts_file}'},
    {'type': 'bind',
     'source': f'./{exec_file}',
     'target': '/app/user_script.py'},
    './selenium-client/result:/app/result']

dc_config['services']['selenium-client']['command'] = [
    "python", "/app/main.py", f"/app/{accounts_file}", gvars['lms_platform_id'],
    "http://selenium-executer:4444/wd/hub", f"-i {lms_tool_id}", "-s/app/user_script.py"]

with Path('tmp/selenium/docker-compose.yml').open(mode='w') as f:
    yaml.safe_dump(dc_config, stream=f)

ダウンロードして利用するため、圧縮します。

In [ ]:
!tar -czf selenium.tgz -C ./tmp/selenium .

作成した仮ファイルを削除します。

In [ ]:
!rm -rf tmp/selenium

### 実行  

先ほど圧縮したファイルを実行するマシンに配置し、解凍した後に起動してください。  
`docker-compose.yml`を指定し、`docker compose up -d`を実行することで、コンテナが起動し、seleniumによる処理が行われます。  
処理結果は、`selenium-client/result`ディレクトリに `.json`形式で保存されます。

#### VCノード以外のマシン上で実行する場合  
ご自身のPC等で実行する場合は、ここまでで作成・圧縮したファイル（`mcj-selenium.tgz`）をダウンロードし、任意のディレクトリで解凍してください。
起動は`docker compose up -d`、終了後は`docker compose down`でコンテナをシャットダウンしてください。

#### VCノード（manager）上で実行する場合

VCノード上で実行する場合は、この章の内容が利用可能です。  
VCノード上の資源を利用してseleniumを動作させるため、検証対象のユーザが利用する資源を消費します。  
そのため、本格的な検証には向きませんが、他マシンのセットアップが不要であるので、簡易に検証を行いたい場合におすすめです。  
現在利用可能なのは、managerノードのみです。

##### 設定

資材を配置するディレクトリを指定してください。 基本的には、デフォルトのままで問題ありません。

In [ ]:
work_dir = '/srv/jupyterhub/spawntest'

##### チェック

対象となるVCノードがAnsibleによって操作できることを確認します。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path('ansible.cfg')
if cfg_ansible.exists():
    os.environ['ANSIBLE_CONFIG'] = str(cfg_ansible.resolve())

In [ ]:
target_hub = f'{ugroup_name}_manager'

!ansible {target_hub} -m ping

テスト対象のサーバにseleniumがアクセスできることを確認します。

In [ ]:
# selenium実行環境からmoodleへのアクセス
!ansible {target_hub} -m shell \
    -a "curl {{{{lms_platform_id}}}} -o /dev/null -w '%{{http_code}}\n' -s"
# selenium実行環境からjupyterhubへのアクセス
!ansible {target_hub} -m shell \
    -a "curl https://{{{{jupyterhub_fqdn}}}}/hub/health -o /dev/null -w '%{{http_code}}\n' -s"

##### ファイル配置

設定ファイルを配置・展開します。

In [ ]:
!ansible {target_hub} -m file \
    -a "state=directory path={work_dir}/selenium"

!ansible {target_hub} -m unarchive \
    -a "src=selenium.tgz dest={work_dir}/selenium"

##### 実行

In [ ]:
!ansible {target_hub} -m shell \
    -a 'chdir={work_dir}/selenium/ docker-compose up -d --build'

実行完了まで待機します。

In [ ]:
import time
MAX_RETRY = 30
INTERVAL = 10
for i in range(MAX_RETRY):
    active_user = !ansible {target_hub} -m shell \
        -a 'docker ps | grep -c selenium-client || true'
    if int(active_user[1]) == 0:
        print('Selenium client finished.')
        break
    
    print('Waiting selenium finish.')
    time.sleep(INTERVAL)
else:
    raise Exception(f'Not finished in {INTERVAL*MAX_RETRY} secondes.')

結果を確認します。  
以下のセルを実行すると、selenium実行結果ファイルのうち最新のものの内容を表示します。

In [ ]:
!ansible {target_hub} -m shell \
    -a 'ls -rt {work_dir}/selenium/selenium-client/result/* | tail -n 1 | xargs cat'

結果が出力されない場合、アクセス自体出来ていない可能性があります。  
そのような場合、以下のセルを実行し、selenium実行コンテナのログを確認します。  
「Failed to connect」等のログがある場合、seleniumを実行している環境からLMSへのアクセス許可等の確認を行ってください。

In [ ]:
!ansible {target_hub} -m shell \
    -a 'docker logs selenium-executer'

seleniumクライアントコンテナのログを確認します。  
selenium実行時の、各ユーザのログが出力されます。

In [ ]:
!ansible {target_hub} -m shell \
    -a 'docker logs selenium-client'

##### 終了  

コンテナをシャットダウンします。

In [ ]:
!ansible {target_hub} -m shell \
    -a 'chdir={work_dir}/selenium/ docker-compose down'
!ansible {target_hub} -m shell \
    -a 'chdir={work_dir}/selenium/ docker-compose rm'